# Tool system design

When AI agents need to interact with the outside world such as querying databases, calling APIs, performing calculations or accessing real-time information, they rely on tools. While tool integration extends agent capabilities dramatically, poorly designed tools can become a liability. Ambiguous names confuse agents about which tool to use, unclear parameter specifications lead to errors, and verbose descriptions waste precious context window space. The result is agents that struggle to select the right tool, invoke it correctly or understand what it returns.

Tool system design is fundamentally a context engineering challenge. Every tool name, parameter definition, and description contributes to the context that guides the agent's behavior. Well-designed tools act as clear, self-documenting interfaces that maximize the agent's ability to solve problems while minimizing the tokens required to explain how. This means choosing names that immediately convey purpose, structuring parameters with validation that prevents errors before they occur, and writing descriptions that are concise yet comprehensive.

In this notebook, we explore techniques for designing tool systems that agents can use reliably and efficiently. We will examine how to establish clear naming conventions that reflect tool purpose, structure parameters using self-documenting Pydantic schemas, write concise yet comprehensive descriptions, include strategic usage examples, organize tools into logical categories, and maximize information density while minimizing token usage.

In [1]:
import os
from typing import Optional, List, Dict
from pydantic import BaseModel, Field, field_validator
from enum import Enum
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

### Initialize the language model
We initialize the OpenAI chat model.

In [2]:
# Initialize the language model
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    api_key=os.getenv("OPENAI_API_KEY", "").strip(),
    temperature=0  # Set to 0 for more deterministic outputs
)

- **`temperature=0`**: Ensures the model returns the most likely response, making it consistent and reproducible.

## Part 1: Clear naming conventions
Tool names are the first filter agents use when selecting which tool to invoke. A well-named tool communicates its purpose instantly, while a poorly named tool forces the agent to read full descriptions or worse, guess incorrectly. The consequences of bad naming compound across large tool sets - agents waste tokens processing irrelevant tool descriptions, make incorrect tool selections, and require additional context to disambiguate similar-sounding tools.

Effective naming follows consistent patterns that agents can learn and predict. The action-object pattern uses a verb followed by the target, making the tool's purpose immediately clear. Specificity prevents ambiguity about what the tool actually does, while brevity keeps names concise and memorable. Consistency across related tools allows agents to infer functionality from naming patterns alone.

### Poor naming conventions
Let's look at examples of names that confuse the model:

In [3]:
# Examples of poor tool names
bad_names = [
    "tool1",  # ❌ No semantic meaning; agent cannot infer purpose
    "get_data",  # ❌ Too vague - what kind of data? User data? Product data?
    "process",  # ❌ Ambiguous verb - process what? A payment? An image?
    "handle_user_thing",  # ❌ Unprofessional and unclear scope
    "database_query_executor_with_validation",  # ❌ Overly verbose; wastes tokens
]

print("Examples of poorly named tools:")
for name in bad_names:
    print(f"  ❌ {name}")

Examples of poorly named tools:
  ❌ tool1
  ❌ get_data
  ❌ process
  ❌ handle_user_thing
  ❌ database_query_executor_with_validation


### Good naming conventions

Effective names follow these principles:
1. **Action + Object**: Use a verb followed by the object (e.g., `search_products`, `send_email`)
2. **Specificity**: Be specific about what the tool does
3. **Brevity**: Keep names concise (2-3 words typically)
4. **Consistency**: Use consistent patterns across related tools

In [4]:
# Examples of well-named tools using Action-Object pattern
good_names = {
    "search_products": "Searches product catalog",  # ✅ Action (search) + Object (products)
    "get_order_status": "Retrieves order status by ID",  # ✅ Specific and clear
    "calculate_shipping": "Computes shipping cost",  # ✅ Unambiguous purpose
    "send_email": "Sends email to recipient",  # ✅ Standard terminology
    "validate_coupon": "Checks coupon code validity",  # ✅ Clear validation action
}

print("Examples of well-named tools:")
print()
for name, description in good_names.items():
    print(f"  ✅ {name}")
    print(f"     {description}")
    print()

Examples of well-named tools:

  ✅ search_products
     Searches product catalog

  ✅ get_order_status
     Retrieves order status by ID

  ✅ calculate_shipping
     Computes shipping cost

  ✅ send_email
     Sends email to recipient

  ✅ validate_coupon
     Checks coupon code validity



These names work because:
- Predictability: The agent can predict what the tool does just by reading the name.
- Specificity: `search_products` is distinct from `search_orders`.
- Brevity: They are concise but descriptive.

### Naming patterns for related tools

When building a large system with many tools, individual good names are not enough; we need system-wide consistency. Use consistent prefixes for related operations:

In [5]:
# Consistent naming patterns
naming_patterns = {
    "Customer operations": [
        "get_customer_info",
        "update_customer_address",
        "delete_customer_account",
    ],
    "Inventory operations": [
        "check_inventory",
        "reserve_inventory",
        "release_inventory",
    ],
    "Payment operations": [
        "process_payment",
        "refund_payment",
        "verify_payment",
    ],
}

print("Consistent naming patterns:")
print()
for category, tool_names in naming_patterns.items():
    print(f"{category}:")
    for tool_name in tool_names:
        print(f"  - {tool_names}")
    print()

Consistent naming patterns:

Customer operations:
  - ['get_customer_info', 'update_customer_address', 'delete_customer_account']
  - ['get_customer_info', 'update_customer_address', 'delete_customer_account']
  - ['get_customer_info', 'update_customer_address', 'delete_customer_account']

Inventory operations:
  - ['check_inventory', 'reserve_inventory', 'release_inventory']
  - ['check_inventory', 'reserve_inventory', 'release_inventory']
  - ['check_inventory', 'reserve_inventory', 'release_inventory']

Payment operations:
  - ['process_payment', 'refund_payment', 'verify_payment']
  - ['process_payment', 'refund_payment', 'verify_payment']
  - ['process_payment', 'refund_payment', 'verify_payment']



As we add more tools, sticking to these patterns ensures the system remains understandable for both the AI and the human developers.

## Part 2: Self-documenting schemas with Pydantic

Parameter specifications are where many tool systems fail. Without clear schemas, agents must guess at parameter types, whether fields are required or optional, what values are valid, and how parameters relate to each other. This leads to errors, failed tool invocations, and wasted context on error handling. Pydantic schemas solve this by making parameter contracts explicit, providing automatic validation, and generating clear documentation that agents can understand.

Self-documenting schemas reduce the need for verbose descriptions because the schema itself communicates structure. Field names, types, default values, validation constraints, and descriptive strings combine to create a complete specification. This shifts documentation from prose to structure, making it both more precise and more token-efficient.

### The problem with implicit schemas
If a tool definition doesn't explicitly state what types it expects (String? Integer?), the LLM has to guess. This guessing leads to runtime errors where the model passes a string "five" instead of the number `5`.

In [6]:
# Tool without schema - parameters are unclear
@tool
def search_products_no_schema(query, category, min_price, max_price, in_stock):
    """Search products. Pass query string, optional category, price range, and stock filter."""
    return f"Searching for: {query}"

print("Tool without schema:")
print(f"Name: {search_products_no_schema.name}")
print(f"Description: {search_products_no_schema.description}")

Tool without schema:
Name: search_products_no_schema
Description: Search products. Pass query string, optional category, price range, and stock filter.


Problems:
  - Parameter types unclear (string? number? boolean?)
  - Which parameters are required vs optional?
  - What are valid values for 'category'?
  - No validation of inputs

When we define a tool like this, LangChain attempts to infer the schema from the function signature. However:
-   Missing types: Without type hints (`: str`, `: int`), the generated JSON Schema defaults to `any` or `string` for all fields. The LLM doesn't know `min_price` should be a number.
-   Missing Required/Optional status: Without default values (`= None`), the LLM might think all parameters are required, forcing it to hallucinate values for `category` even when the user didn't specify one.
-   No validation: The code inside the function receives raw inputs. If `min_price` comes in as "cheap", the code will crash.

### With Pydantic schemas (clear and validated)
To solve the implicit schema problem, we use Pydantic. Pydantic allows us to define a class that represents the arguments for our tool. We can specify types, default values, and even descriptions for each individual argument. This creates a "contract" that the LLM must follow. If the LLM tries to break this contract (e.g., by sending a string for a price), Pydantic intercepts the call and raises an error, which we can then feed back to the agent to correct itself.

In [7]:
# Define a clear schema with Pydantic
class ProductSearchInput(BaseModel):
    """Input schema for product search."""
    # Field descriptions help the LLM understand semantic meaning
    query: str = Field(description="Search term for product name or description")
    # Optional fields with default values
    category: Optional[str] = Field(
        None, 
        description="Product category: electronics, clothing, books, or home"
    )
    # Validation: ge=0 ensures price is non-negative
    min_price: Optional[float] = Field(
        None, 
        ge=0,
        description="Minimum price in USD"
    )
    max_price: Optional[float] = Field(
        None,
        ge=0,
        description="Maximum price in USD"
    )
    in_stock_only: bool = Field(
        True,
        description="Only show in-stock items"
    )

# Bind the schema to the tool using the args_schema parameter
@tool(args_schema=ProductSearchInput)
def search_products(query: str, category: Optional[str] = None, 
                   min_price: Optional[float] = None, max_price: Optional[float] = None,
                   in_stock_only: bool = True) -> str:
    """Search product catalog with filters."""
    # Simulate search logic
    filters = [f"query={query}"]
    if category:
        filters.append(f"category={category}")
    if min_price or max_price:
        filters.append(f"price_range=${min_price or 0}-${max_price or 'unlimited'}")
    if in_stock_only:
        filters.append("in_stock=true")
    return f"Found products: {', '.join(filters)}"

print("Tool with Pydantic schema:")
print(f"Name: {search_products.name}")
print(f"Description: {search_products.description}")
print(f"\nSchema fields:")
for field_name, field_info in ProductSearchInput.model_fields.items():
    required = "required" if field_info.is_required() else "optional"
    print(f"  - {field_name} ({field_info.annotation}, {required}): {field_info.description}")

Tool with Pydantic schema:
Name: search_products
Description: Search product catalog with filters.

Schema fields:
  - query (<class 'str'>, required): Search term for product name or description
  - category (typing.Optional[str], optional): Product category: electronics, clothing, books, or home
  - min_price (typing.Optional[float], optional): Minimum price in USD
  - max_price (typing.Optional[float], optional): Maximum price in USD
  - in_stock_only (<class 'bool'>, optional): Only show in-stock items


- `BaseModel`: This is the parent class for all schemas. It handles the parsing and validation logic.
- `Field(description="...")`: This is the most important part for the LLM. The string passed to `description` is included in the prompt. It tells the model *what* the parameter means. Without this, the model only knows the parameter name.
- `ge=0`: This stands for "greater than or equal to 0". It is a constraint. If the LLM tries to call the tool with `min_price=-5`, Pydantic will raise a validation error *before* the `search_products` function is even called. This protects our code from bad data.
- `@tool(args_schema=...)`: This decorator links the Pydantic model to the function. LangChain uses this link to generate the tool definition for OpenAI.

### Advanced schema features
Basic types like `str` or `int` are often too loose. For example, a "priority" field should not just be any string; it should be one of a few specific options (Low, Medium, High). An "email" field should not just be any text; it must have an `@` symbol.

We can enforce these stricter rules using enums and regex patterns. This guides the LLM even further, narrowing down the possible outputs it can generate. The goal is to make it impossible for the model to provide invalid options.

In [8]:
# Use enums for constrained choices - LLM can only pick from these
class Priority(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    URGENT = "urgent"

class TicketInput(BaseModel):
    """Input schema for creating support tickets."""
    # Length constraints
    title: str = Field(..., min_length=5, max_length=100, description="Brief ticket title")
    description: str = Field(..., min_length=20, description="Detailed description of the issue")
    
    # Enum constraint: The model MUST choose one of the Priority values
    priority: Priority = Field(Priority.MEDIUM, description="Ticket priority level")
    # Regex pattern for email validation
    customer_email: str = Field(..., pattern=r'^[\w\.-]+@[\w\.-]+\.\w+$', description="Customer email address")
    
    tags: List[str] = Field(default_factory=list, description="Optional tags for categorization")

    # Custom validator method to enforce complex logic
    @field_validator('tags')
    def validate_tags(cls, v):
        if len(v) > 5:
            raise ValueError('Maximum 5 tags allowed')
        return v

# Bind the schema to the tool using the args_schema parameter
@tool(args_schema=TicketInput)
def create_support_ticket(title: str, description: str, priority: Priority = Priority.MEDIUM,
                         customer_email: str = "", tags: List[str] = None) -> str:
    """Create a new support ticket."""
    tags = tags or []
    return f"Ticket created: {title} (Priority: {priority.value}, Email: {customer_email})"

print("Advanced schema features:")
print(f"\nEnum for priority: {[p.value for p in Priority]}")
print(f"Email validation: regex pattern enforced")
print(f"String length constraints: title must be 5-100 chars")
print(f"Custom validation: max 5 tags")

Advanced schema features:

Enum for priority: ['low', 'medium', 'high', 'urgent']
Email validation: regex pattern enforced
String length constraints: title must be 5-100 chars
Custom validation: max 5 tags


- `Enum` (Enumeration): By defining `Priority` as an Enum, we tell the LLM that `priority` is not an open-ended string. It must be one of the defined values (`low`, `medium`, etc.). This eliminates typos and invalid categories.
- `pattern` (Regex): The `customer_email` field uses a regular expression. If the LLM generates "john.doe" (missing the @domain), Pydantic catches this error immediately.
- `@field_validator`: This decorator allows us to write arbitrary Python code to validate fields. Here, we check that the list of tags does not exceed 5 items. This kind of logic is hard to express in standard JSON Schema but easy in Pydantic.

## Part 3: Concise yet comprehensive descriptions
Tool descriptions must accomplish two goals simultaneously: explain what the tool does and indicate when to use it, all while consuming minimal tokens. Too vague and agents misuse the tool; too verbose and we waste context window. The solution is a structured approach that prioritizes the most important information.

Effective descriptions follow a simple pattern: start with a clear action statement in 5-7 words, optionally add one sentence of context about when to use the tool, and include critical constraints only when they affect tool selection. This framework ensures descriptions provide necessary guidance without redundancy.

In [9]:
# Examples of description quality levels

# ❌ Too vague
@tool
def tool_vague(customer_id: str) -> str:
    """Gets information."""
    return f"Info for {customer_id}"

# ❌ Too verbose
@tool
def tool_verbose(customer_id: str) -> str:
    """This tool is used to retrieve comprehensive customer information from our database.
    You should use this tool whenever you need to access details about a specific customer,
    such as when answering questions about their account, order history, or preferences.
    The tool accepts a customer ID as input and returns all available information including
    name, email, phone, address, and order history. Note that you must have the customer ID
    before calling this tool, which can be obtained from other tools or user input."""
    return f"Info for {customer_id}"

# ✅ Just right
@tool
def get_customer_info(customer_id: str) -> str:
    """Retrieve customer details by ID. Returns name, contact info, and order history."""
    return f"Info for {customer_id}"

print("Description comparison:")
print(f"\n❌ Too vague ({len(tool_vague.description)} chars):")
print(f"   '{tool_vague.description}'")
print(f"\n❌ Too verbose ({len(tool_verbose.description)} chars):")
print(f"   '{tool_verbose.description[:100]}...'")
print(f"\n✅ Just right ({len(get_customer_info.description)} chars):")
print(f"   '{get_customer_info.description}'")

Description comparison:

❌ Too vague (17 chars):
   'Gets information.'

❌ Too verbose (532 chars):
   'This tool is used to retrieve comprehensive customer information from our database.
    You should u...'

✅ Just right (79 chars):
   'Retrieve customer details by ID. Returns name, contact info, and order history.'


#### Building a suite of well-described tools

In [10]:
# Well-designed tool suite for e-commerce

class OrderStatusInput(BaseModel):
    order_id: str = Field(description="Order ID to check")

@tool(args_schema=OrderStatusInput)
def get_order_status(order_id: str) -> str:
    """Check order status and tracking. Use when customer asks about order location or delivery."""
    return f"Order {order_id}: Shipped, arrives Tuesday"

class ShippingInput(BaseModel):
    zipcode: str = Field(description="Destination ZIP code")
    weight_lbs: float = Field(description="Package weight in pounds", gt=0)
    expedited: bool = Field(False, description="Use expedited shipping")

@tool(args_schema=ShippingInput)
def calculate_shipping(zipcode: str, weight_lbs: float, expedited: bool = False) -> str:
    """Calculate shipping cost for destination and weight. Use before checkout."""
    cost = weight_lbs * (4 if expedited else 2)
    return f"Shipping to {zipcode}: ${cost:.2f} ({'2-day' if expedited else '5-7 day'} delivery)"

class InventoryInput(BaseModel):
    product_id: str = Field(description="Product SKU or ID")
    zipcode: Optional[str] = Field(None, description="ZIP code for local availability")

@tool(args_schema=InventoryInput)
def check_inventory(product_id: str, zipcode: Optional[str] = None) -> str:
    """Check product availability. Use before purchase to confirm stock."""
    location = f" near {zipcode}" if zipcode else " online"
    return f"Product {product_id}: 15 units available{location}"

class RefundInput(BaseModel):
    order_id: str = Field(description="Order to refund")
    reason: str = Field(description="Refund reason")

@tool(args_schema=RefundInput)
def process_refund(order_id: str, reason: str) -> str:
    """Initiate refund for order. Only use if within 30-day policy and customer requests refund."""
    return f"Refund initiated for {order_id}. Reason: {reason}. Processed in 5-7 days."

# Display tool descriptions
example_tools = [get_order_status, calculate_shipping, check_inventory, process_refund]

print("Well-designed tool suite:")
print()
for example_tool in example_tools:
    print(f"📦 {example_tool.name}")
    print(f"   Description: {example_tool.description}")
    print(f"   Characters: {len(example_tool.description)}")
    print()

Well-designed tool suite:

📦 get_order_status
   Description: Check order status and tracking. Use when customer asks about order location or delivery.
   Characters: 89

📦 calculate_shipping
   Description: Calculate shipping cost for destination and weight. Use before checkout.
   Characters: 72

📦 check_inventory
   Description: Check product availability. Use before purchase to confirm stock.
   Characters: 65

📦 process_refund
   Description: Initiate refund for order. Only use if within 30-day policy and customer requests refund.
   Characters: 89



## Part 4: Including usage examples
Sometimes, a field description isn't enough. For complex formats like dates or special codes, the best way to explain is by example. We can include examples directly in the docstring. This acts as few-shot prompting right inside the tool definition. The LLM sees the example usage and mimics the format. The goal is to reduce ambiguity for complex parameters.

In [11]:
class DateRangeInput(BaseModel):
    start_date: str = Field(description="Start date (YYYY-MM-DD)")
    end_date: str = Field(description="End date (YYYY-MM-DD)")
    category: Optional[str] = Field(None, description="Filter by product category")

@tool(args_schema=DateRangeInput)
def get_sales_report(start_date: str, end_date: str, category: Optional[str] = None) -> str:
    """Generate sales report for date range.
    
    Example: To get electronics sales for January 2024:
    start_date="2024-01-01", end_date="2024-01-31", category="electronics"
    
    Returns: Total sales, units sold, top products.
    """
    return f"Sales report: {start_date} to {end_date}, category: {category or 'all'}"

print("Tool with usage example:")
print(get_sales_report.description)

Tool with usage example:
Generate sales report for date range.

    Example: To get electronics sales for January 2024:
    start_date="2024-01-01", end_date="2024-01-31", category="electronics"

    Returns: Total sales, units sold, top products.


Benefits of including examples:
  - Clarifies date format requirements
  - Shows how parameters work together
  - Illustrates a concrete use case
  - Hints at expected output

The docstring here serves a dual purpose:
- Human readability: Developers reading the code understand how to use the function.
- LLM prompting: LangChain extracts this docstring and sends it to the model. The section `Example: ...` provides a concrete pattern for the model to follow. When the model sees `start_date="2024-01-01"`, it learns the expected date format (ISO 8601) without us needing to write a complex regex.

## Part 5: Organizing tools into logical categories
As our agent grows, we might end up with 50 or 100 tools. If we simply pass a flat list of 100 tools to the LLM, we will overwhelm it. This "cognitive overload" leads to the model forgetting tools or hallucinating. The goal of this section is to visualize the problem of a flat list versus a structured approach.

### Without organization

In [12]:
# Unorganized tool list (harder to navigate)
unorganized_tools = [
    "send_email", "get_inventory", "update_customer", "calculate_tax",
    "process_return", "send_sms", "check_stock", "verify_address",
    "apply_discount", "send_notification", "reserve_stock"
]

print("Unorganized tool list (flat structure):")
for unorganized_tool in unorganized_tools:
    print(f"  - {unorganized_tool}")
print("\n⚠️ Agent must scan all 11 tools for every decision")

Unorganized tool list (flat structure):
  - send_email
  - get_inventory
  - update_customer
  - calculate_tax
  - process_return
  - send_sms
  - check_stock
  - verify_address
  - apply_discount
  - send_notification
  - reserve_stock

⚠️ Agent must scan all 11 tools for every decision


In a flat list, the attention mechanism of the Transformer model has to attend to every single tool definition equally. This dilutes the attention scores. By contrast, if we organize tools, we can help the model focus.

### With logical organization
Group related tools to reduce decision complexity and make tool selection clearer for the agent. We group tools into categories. This allows for a two-step reasoning process (though often implicit): First, "I need to do something with Inventory", then "I will look at the Inventory tools". This logical grouping helps the model narrow down its search space.

In [13]:
# Organized into categories
organized_tools = {
    "Communication": ["send_email", "send_sms", "send_notification"],
    "Inventory": ["get_inventory", "check_stock", "reserve_stock"],
    "Customer": ["update_customer", "verify_address"],
    "Orders": ["process_return", "apply_discount", "calculate_tax"],
}

print("Organized tool structure (by category):")
for category, tools in organized_tools.items():
    print(f"\n{category}:")
    for organized_tool in tools:
        print(f"  - {organized_tool}")

print("\n✅ Agent can narrow down category first, then select specific tool")

Organized tool structure (by category):

Communication:
  - send_email
  - send_sms
  - send_notification

Inventory:
  - get_inventory
  - check_stock
  - reserve_stock

Customer:
  - update_customer
  - verify_address

Orders:
  - process_return
  - apply_discount
  - calculate_tax

✅ Agent can narrow down category first, then select specific tool


While the `organized_tools` dictionary above is just a Python structure, in a real agent system, we might actually use separate agents or "routers" for each category. For example, a "Supervisor" agent might route the request to a "Inventory Specialist" agent that only has access to the Inventory tools. This is known as a multi-agent architecture.

### Implementing categorization in code
In a production codebase, we rarely just have loose functions. We want to attach metadata to our tools, such as which category they belong to, or what permissions are required to use them. We can create a `CategorizedTool` wrapper. This allows us to manage our tools as rich objects rather than just functions.

In [14]:
# Define tools with category metadata
class CategorizedTool:
    def __init__(self, name: str, description: str, category: str, function):
        self.name = name
        self.description = description
        self.category = category
        self.function = function

# Create categorized tools
categorized_tool_list = [
    CategorizedTool(
        "send_email",
        "Send email to customer. Use for order confirmations, updates, or support.",
        "Communication",
        lambda to, subject, body: f"Email sent to {to}"
    ),
    CategorizedTool(
        "check_inventory",
        "Check product stock levels. Use before confirming orders.",
        "Inventory",
        lambda product_id: f"Stock for {product_id}: 50 units"
    ),
    CategorizedTool(
        "process_refund",
        "Initiate customer refund. Use only if within return policy.",
        "Orders",
        lambda order_id: f"Refund processed for {order_id}"
    ),
]

# Group by category
def get_tools_by_category(tools: List[CategorizedTool], category: str) -> List[CategorizedTool]:
    return [t for t in tools if t.category == category]

print("Tool selection by category:")
print()
print("Available categories:", set(t.category for t in categorized_tool_list))
print()
print("Communication tools:")
for tool_by_category in get_tools_by_category(categorized_tool_list, "Communication"):
    print(f"  - {tool_by_category.name}: {tool_by_category.description}")

Tool selection by category:

Available categories: {'Orders', 'Inventory', 'Communication'}

Communication tools:
  - send_email: Send email to customer. Use for order confirmations, updates, or support.


The `CategorizedTool` class acts as a container. 
- Metadata storage: It holds the `category` string alongside the function.
- Dynamic filtering: The `get_tools_by_category` function demonstrates how we can dynamically filter the tool set. In a chatbot, if the user says "I have a billing question", we could programmatically filter for only "Orders" or "Payment" tools and pass only those to the LLM, saving tokens and improving accuracy.

## Part 6: Maximally informative with minimal tokens
Bringing it all together: create tools that provide maximum clarity with minimum token usage.

### Before: Verbose and unclear
The legacy approach relied on putting everything into a massive docstring. This was common in early LangChain versions. However, it is verbose, hard to read, and prone to parsing errors. The goal here is to visually contrast this with the clean, structured Pydantic approach.

In [15]:
@tool
def old_product_search(search_term, product_category, lowest_price, highest_price, only_in_stock):
    """This is a tool that you can use to search through all the products in our catalog.
    You should use this tool whenever a customer asks about finding products or wants to browse
    our inventory. The tool takes several parameters: search_term is what to look for,
    product_category filters by category, lowest_price and highest_price set price bounds,
    and only_in_stock determines if we show out-of-stock items. The tool will return a list
    of matching products with their details including name, price, and availability."""
    return "Found products"

print("❌ Old approach:")
print(f"Description length: {len(old_product_search.description)} characters")
print(f"Parameter names: Unclear (search_term vs query, lowest_price vs min_price)")
print(f"No type information or validation")

❌ Old approach:
Description length: 533 characters
Parameter names: Unclear (search_term vs query, lowest_price vs min_price)
No type information or validation


The `old_product_search` tool relies on the LLM's ability to parse natural language instructions to understand arguments. 
- Token waste: The description is 300+ characters long. 
- Ambiguity: "lowest_price" is mentioned in text, but is it a float? An integer? The model has to guess.
- Fragility: If we change a parameter name in the function but forget to update the text description, the tool breaks.

### After: Concise and clear with schema
By using Pydantic, we strip away the verbose text. The structure is the documentation. This is cleaner, safer, and cheaper (in terms of tokens).

In [16]:
class ProductSearchSchema(BaseModel):
    """Search products with filters."""
    query: str = Field(description="Search term")
    category: Optional[str] = Field(None, description="Filter by category")
    min_price: Optional[float] = Field(None, ge=0, description="Min price USD")
    max_price: Optional[float] = Field(None, ge=0, description="Max price USD")
    in_stock_only: bool = Field(True, description="Exclude out-of-stock")

@tool(args_schema=ProductSearchSchema)
def search_products_new(query: str, category: Optional[str] = None,
                       min_price: Optional[float] = None, max_price: Optional[float] = None,
                       in_stock_only: bool = True) -> str:
    """Search product catalog with filters."""
    return "Found products"

print("✅ New approach:")
print(f"Description length: {len(search_products_new.description)} characters")
print(f"Parameter names: Clear and consistent")
print(f"Schema provides: Types, defaults, validation, field descriptions")
print(f"\nToken savings: {len(old_product_search.description) - len(search_products_new.description)} characters")

✅ New approach:
Description length: 36 characters
Parameter names: Clear and consistent
Schema provides: Types, defaults, validation, field descriptions

Token savings: 497 characters


The `search_products_new` tool is superior because:
- Self-documenting: The code itself defines the interface.
- Token savings: We saved over 400 characters of description text. In a long conversation, this adds up significantly.
- Type safety: We have explicit `float` types and `ge=0` constraints.